In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os

import torch
from torch.utils.data import DataLoader, random_split
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision import transforms
from torchvision.models import resnet50
import numpy as np
from tqdm import tqdm

from LoadData import CustomDataset
from LabelSmoothing import LabelSmoothingLoss

In [ ]:
### Hyperparameters ###
re_size = 256
crop_size = 224
batch_size = 64
lr_begin = (batch_size / 256) * 0.1  # learning rate at begining
epochs = 60
seed = 0

In [ ]:
### Random seeds ###
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)

np.random.seed(seed)

In [ ]:
### Define Transforms ###
train_transform = transforms.Compose(
    [
        transforms.Resize((re_size, re_size)),
        transforms.RandomCrop(crop_size, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
val_transform = transforms.Compose(
    [
        transforms.Resize((re_size, re_size)),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [ ]:
def custom_collate_train(batch):
    images, labels = zip(*batch)
    images = [train_transform(img) for img in images]
    return torch.stack(images), torch.tensor(labels)
def custom_collate_val(batch):
    images, labels = zip(*batch)
    images = [val_transform(img) for img in images]
    return torch.stack(images), torch.tensor(labels)

### Load Datasets ###
dataset = CustomDataset(root_dir=os.path.join(os.getcwd(), 'gdrive/MyDrive/ML_Final/data/train'))

# Split into training and validation
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, collate_fn=custom_collate_train)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, collate_fn=custom_collate_val)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
### Model Setting ###
net = resnet50(pretrained=True)
net.fc = torch.nn.Linear(net.fc.in_features, 200)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)
print(device)

for param in net.parameters():
    param.requires_grad = True

criterion = LabelSmoothingLoss(classes=200, smoothing=0.1)  # label smoothing to improve performance
optimizer = torch.optim.SGD(net.parameters(), lr=lr_begin, momentum=0.9, weight_decay=5e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=epochs)

# Check if a pre-trained model checkpoint exists
pretrained_model_path = os.path.join(os.getcwd(), 'gdrive/MyDrive/ML_Final/pretrained_model.pth')
if os.path.isfile(pretrained_model_path):
    # Load the model checkpoint
    checkpoint = torch.load(pretrained_model_path)

    # Load the model and optimizer states
    net.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    print(f"Loaded pre-trained model.")
else:
    print(f"Training from scratch.")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 77.1MB/s]


cuda
Training from scratch.


In [ ]:
previous_losses = os.path.join(os.getcwd(), 'gdrive/MyDrive/ML_Final/train_losses.npy')
previous_val_losses = os.path.join(os.getcwd(), 'gdrive/MyDrive/ML_Final/val_losses.npy')
previous_acc = os.path.join(os.getcwd(), 'gdrive/MyDrive/ML_Final/train_acc.npy')
previous_val_acc = os.path.join(os.getcwd(), 'gdrive/MyDrive/ML_Final/val_acc.npy')

if os.path.exists(previous_losses):
    train_losses = np.load(previous_losses)
    val_losses = np.load(previous_val_losses)
    train_accs = np.load(previous_acc)
    val_accs = np.load(previous_val_acc)
    print("Load previous Info.")
else:
    train_losses = np.array([])
    val_losses = np.array([])
    train_accs = np.array([])
    val_accs = np.array([])

for epoch in range(epochs):
    print('\n===== Epoch: {} ====='.format(epoch))
    ### Training ###
    net.train()

    train_loss = train_correct = train_total = idx = 0
    for batch_idx, (inputs, targets) in enumerate(tqdm(train_loader, ncols=80)):
        idx = batch_idx

        optimizer.zero_grad()  # Sets the gradients to zero
        inputs, targets = inputs.to(device), targets.to(device)

        x = net(inputs)
        loss = criterion(x, targets)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(x.data, 1)
        train_total += targets.size(0)
        train_correct += predicted.eq(targets.data).cpu().sum()
        train_loss += loss.item()

    scheduler.step()

    train_acc = 100.0 * float(train_correct) / train_total
    train_loss = train_loss / (idx + 1)
    train_losses = np.append(train_losses, train_loss)
    train_accs = np.append(train_accs, train_acc)

    print('Train | Loss: {:.4f} | Acc: {:.3f}% ({}/{})'.format(train_loss, train_acc, train_correct, train_total))

    ### Validation ###
    net.eval()  # set model to evaluation mode

    val_loss = val_correct = val_total = idx = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(tqdm(val_loader, ncols=80)):
            idx = batch_idx

            inputs, targets = inputs.to(device), targets.to(device)

            x = net(inputs)
            loss = criterion(x, targets)

            _, predicted = torch.max(x.data, 1)
            val_total += targets.size(0)
            val_correct += predicted.eq(targets.data).cpu().sum()
            val_loss += loss.item()

    val_acc = 100.0 * float(val_correct) / val_total
    val_loss = val_loss / (idx + 1)
    val_losses = np.append(val_losses, val_loss)
    val_accs = np.append(val_accs, val_acc)

    print('Validation | Loss: {:.4f} | Acc: {:.3f}% ({}/{})'.format(val_loss, val_acc, val_correct, val_total))

    ### Save Info. ###
    torch.save({
      'model_state_dict': net.state_dict(),
      'optimizer_state_dict': optimizer.state_dict()},
      pretrained_model_path)
    np.save(previous_losses, train_losses)
    np.save(previous_val_losses, val_losses)
    np.save(previous_acc, train_accs)
    np.save(previous_val_acc, val_accs)


===== Epoch: 0 =====


100%|█████████████████████████████████████████| 123/123 [11:52<00:00,  5.79s/it]


Train | Loss: 3.4357 | Acc: 30.562% (2393/7830)


100%|███████████████████████████████████████████| 31/31 [02:55<00:00,  5.65s/it]


Validation | Loss: 3.0750 | Acc: 36.057% (706/1958)

===== Epoch: 1 =====


...

Validation | Loss: 1.5208 | Acc: 84.474% (1654/1958)

===== Epoch: 59 =====


100%|█████████████████████████████████████████| 123/123 [01:22<00:00,  1.49it/s]


Train | Loss: 0.8775 | Acc: 100.000% (7830/7830)


100%|███████████████████████████████████████████| 31/31 [00:12<00:00,  2.45it/s]


Validation | Loss: 1.5230 | Acc: 84.321% (1651/1958)
